# File manager by Multi-thread processing

## Import necessary libraries:

In [1]:
import os
import shutil
import threading
import numpy as np
import tensorflow as tf
import script.ImageProcessing as IP
from tensorflow.keras.models import load_model
from concurrent.futures import ThreadPoolExecutor

In [2]:
# Avoid out of memory errors by setting GPU Memory Consumption Growth
gpus = tf.config.experimental.list_physical_devices('GPU')
for gpu in gpus: 
    tf.config.experimental.set_memory_growth(gpu, True)

## Load your trained Keras model:

Load the Keras model that you have trained for image classification.

In [3]:
MODEL = load_model('.\export model\Res2Net50_224x224_2Class_fp16 (ILLUST VS MANGA).h5')  # Load your trained Keras model
IMAGE_SIZE = MODEL.layers[0].input_shape[0][1]


INFO:tensorflow:Mixed precision compatibility check (mixed_float16): OK
Your GPU will likely run quickly with dtype policy mixed_float16 as it has compute capability of at least 7.0. Your GPU: NVIDIA GeForce RTX 2060, compute capability 7.5


## Define a function to classify images and move them:

Create a function that takes an image path as input, classifies it using your Keras model, and then moves the image to the corresponding folder based on the classification result.

In [4]:
def classify_and_move(image_path, model, image_size):
    image = IP.image_resize_square(image_path, image_size, aspect_ratio=True, padding=True)
    image = np.expand_dims(image.numpy(), axis=0)
    
    prediction = model.predict(image)
    predicted_class = np.argmax(prediction)

    class_name = f"class_{predicted_class}"
    destination_folder = os.path.join(DIR, class_name)
    os.makedirs(destination_folder, exist_ok=True)

    image_filename = os.path.basename(image_path)
    new_image_path = os.path.join(destination_folder, image_filename)
    shutil.move(image_path, new_image_path)

## Collect image paths and create threads:

Traverse through the directory containing the images you want to classify and organize. For each image, create a thread that will run the classify_and_move function.

In [5]:
def organize_images(input_folder, model, image_size):
    image_paths = [os.path.join(input_folder, filename) for filename in os.listdir(input_folder)]

    max_threads = os.cpu_count()  # Maximum number of concurrent threads
    with ThreadPoolExecutor(max_threads) as executor:
        for image_path in image_paths:
            executor.submit(classify_and_move, image_path, model, image_size)


## Execute

In [6]:
DIR = "Directory"  # Folder containing unorganized images
organize_images(DIR, MODEL, IMAGE_SIZE)